In [9]:
import numpy as np
import pandas as pd

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

In [172]:
pd.set_option('display.max_columns', 60)

In [11]:
sns.set_style('darkgrid')
%matplotlib inline

## Завантажимо наші таблички та поглянемо на дані

In [19]:
PATH_TO_DATA = 'data/'

In [24]:
data1 = pd.read_excel(PATH_TO_DATA + 'data1.xlsx', index_col=0)

In [25]:
data1

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Q1,7,9,1,3,6,6,2,9,7,4,3,6,8,9,9,9,5,1,2,5
Q2,0,9,7,9,9,3,2,4,3,7,1,2,2,0,2,1,7,5,1,7


****

In [26]:
data2 = pd.read_excel(PATH_TO_DATA + 'data2.xlsx', index_col=0)

In [27]:
data2

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Q1,7,9,4,5,3,2,3,4,5,3,9,2,8,1,1,2,3,3,9,4
Q2,4,1,7,1,1,1,7,0,4,0,8,5,1,6,6,2,1,9,6,4


****

In [28]:
data3 = pd.read_excel(PATH_TO_DATA + 'data3.xlsx', index_col=0)

In [29]:
data3

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Q1,3,8,1,2,7,6,1,7,7,8,3,4,7,8,7,5,5,4,9,1
Q2,8,0,8,1,0,2,2,9,7,5,6,4,6,3,7,9,7,4,9,1


****

## Сфорумуємо табличку з 60 значеннями альтернатив

In [32]:
data4 = pd.concat([data1,
                   data2.rename(columns={last_col: 'A'+str(int(last_col[1:])+20) for last_col in data2.columns}),
                   data3.rename(columns={last_col: 'A'+str(int(last_col[1:])+40) for last_col in data3.columns})
                  ], axis=1)

In [41]:
data4

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A51,A52,A53,A54,A55,A56,A57,A58,A59,A60
Q1,7,9,1,3,6,6,2,9,7,4,...,3,4,7,8,7,5,5,4,9,1
Q2,0,9,7,9,9,3,2,4,3,7,...,6,4,6,3,7,9,7,4,9,1


## Реалізуємо критерії Парето та Слейтера

In [105]:
def pareto_metric(x1, x2):
    if not isinstance(x1, np.ndarray): x1 = np.array(x1)
    if not isinstance(x2, np.ndarray): x2 = np.array(x2)
    
    return True if (x1 >= x2).all() and (x1 > x2).any() else False

In [106]:
def slater_metric(x1, x2):
    if not isinstance(x1, np.ndarray): x1 = np.array(x1)
    if not isinstance(x2, np.ndarray): x2 = np.array(x2)
        
    return True if (x1 > x2).all() else False

## Реалізуємо пошук відповідних множин в наших даних

In [132]:
def find_next_j(j, N, skip_alternative_list):
    j += 1
    while j < N and skip_alternative_list[j]:
        j += 1
    return j

In [186]:
def compute_criterion(data, criterion=pareto_metric):
    assert isinstance(data, pd.DataFrame)
    
    N = data.shape[1]
    criterion_result = np.array([''] * N, dtype=np.dtype('U100'))
    skip_alternative_list = [False] * N
    
    # 1
    N -= 1
    i, j = 0, 1
    
    while True:
        # 2
        if criterion(data[f'A{i+1}'], data[f'A{j+1}']) and not skip_alternative_list[j]:
            criterion_result[criterion_result == f'A{j+1}'] = f'A{i+1}'
            criterion_result[j] = f'A{i+1}'
            
            # 3
            skip_alternative_list[j] = True
        # 5
        elif criterion(data[f'A{j+1}'], data[f'A{i+1}']) and not skip_alternative_list[i]:
            criterion_result[criterion_result == f'A{i+1}'] = f'A{j+1}'
            criterion_result[i] = f'A{j+1}'
            
            # 6
            skip_alternative_list[i] = True
        # 4
        if j < N:
            j = find_next_j(j, N, skip_alternative_list)
            continue
        # 7
        if i < N - 1:
            i += 1
            j = i + 1
        else:
            break
            
    return criterion_result

In [187]:
def compute_all_metrics(data):
    pareto_results = pd.DataFrame([compute_criterion(data, criterion=pareto_metric)],
                                  columns=[f'A{i+1}' for i in range(data.shape[1])],
                                  index=['Pareto'])
    slater_results = pd.DataFrame([compute_criterion(data, criterion=slater_metric)],
                                  columns=[f'A{i+1}' for i in range(data.shape[1])],
                                  index=['Slater'])
    
    return pd.concat([data, pareto_results, slater_results])

In [188]:
result1 = compute_all_metrics(data1)
result1

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Q1,7,9,1,3,6,6,2,9,7,4,3,6,8,9,9,9,5,1,2,5
Q2,0,9,7,9,9,3,2,4,3,7,1,2,2,0,2,1,7,5,1,7
Pareto,A2,,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2
Slater,A2,,A2,,,A2,A2,,A2,A2,A2,A2,A2,,,,A2,A2,A2,A2


In [189]:
result2 = compute_all_metrics(data2)
result2

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Q1,7,9,4,5,3,2,3,4,5,3,9,2,8,1,1,2,3,3,9,4
Q2,4,1,7,1,1,1,7,0,4,0,8,5,1,6,6,2,1,9,6,4
Pareto,A11,A11,A11,A11,A11,A11,A11,A11,A11,A11,,A3,A2,A3,A3,A1,A1,,A11,A1
Slater,A11,,A11,A11,A11,A11,A11,A11,A11,A11,,A3,A11,A3,A3,A1,A1,,,A11


In [190]:
result3 = compute_all_metrics(data3)
result3

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
Q1,3,8,1,2,7,6,1,7,7,8,3,4,7,8,7,5,5,4,9,1
Q2,8,0,8,1,0,2,2,9,7,5,6,4,6,3,7,9,7,4,9,1
Pareto,A19,A19,A19,A19,A19,A19,A19,A19,A19,A19,A1,A19,A19,A19,A19,A19,A19,A19,,A1
Slater,A8,A19,A8,A8,A19,A8,A8,,A19,A19,A8,A8,A19,A19,A19,,A8,A8,,A1


In [191]:
result4 = compute_all_metrics(data4)
result4

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A29,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A45,A46,A47,A48,A49,A50,A51,A52,A53,A54,A55,A56,A57,A58,A59,A60
Q1,7,9,1,3,6,6,2,9,7,4,3,6,8,9,9,9,5,1,2,5,7,9,4,5,3,2,3,4,5,3,9,2,8,1,1,2,3,3,9,4,3,8,1,2,7,6,1,7,7,8,3,4,7,8,7,5,5,4,9,1
Q2,0,9,7,9,9,3,2,4,3,7,1,2,2,0,2,1,7,5,1,7,4,1,7,1,1,1,7,0,4,0,8,5,1,6,6,2,1,9,6,4,8,0,8,1,0,2,2,9,7,5,6,4,6,3,7,9,7,4,9,1
Pareto,A2,,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A1,A2,A1,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,A2,,A2
Slater,A2,,A2,,,A2,A2,,A2,A2,A2,A2,A2,,,,A2,A2,A2,A2,A2,,A2,A2,A2,A2,A2,A2,A2,A2,,A2,A2,A2,A2,A2,A2,,,A2,A2,A2,A2,A2,A2,A2,A2,,A2,A2,A2,A2,A2,A2,A2,,A2,A2,,A2
